Alright. Another slap at doing tla.


Do i want to overload
== to always be pointwise equality? Uhhh, I dunno.
And then have a coercsion using Valid from anything to Bool using forall.


I could overload fundamentally
If could be overloaded
Implies
And
Or

Overloading is almost counter to coercing. And(a,b,c) coerfce would try to coerce to bools, probably by forall. Overload makes it

prove could auto insert Valid

Instead of typing annotations, could do the microlean thing of requiring delcaration outside the text.
Maybe _just_ formula should be parsed. Hmm.... Yes, that would be quite a bit simpler and make more sense.

But also being able to ingest an entire file is useful. Perhaps the parse could take in an initial declarations dictionary.


https://proofsandintuitions.net/2026/02/09/distributed-verification-veil/


In [9]:
text = r"""
---------------------- MODULE HourClock ----------------------
EXTENDS Naturals
VARIABLE hr
HCini  ==  hr \in (1 .. 12)
HCnxt  ==  hr' = IF hr # 12 THEN hr + 1 ELSE 1
HC  ==  HCini /\ [][HCnxt]_hr
--------------------------------------------------------------
THEOREM  HC => []HCini
==============================================================
"""

In [8]:
from kdrag.all import *
#from kdrag.theories.logic.temporal import *
from kdrag.modal import *
hr = smt.Array('hr', smt.IntSort(), smt.IntSort())
Valid(Box(PEq(Next(hr), PIf(hr < 12, hr + 1, 0))))

HCini = (hr >= 10) & (hr <= 12)
HCnxt = PEq(Next(hr), PIf(~PEq(hr, 12), hr + 1, 1)) # hr != 12 is not pointwise lifted
HC = HCini & Box(HCnxt)
PImplies(HC, Box(HCini))


Lambda(t0!260,
       Implies(intersection(intersection(Lambda(x0!248,
                                        hr[x0!248] >= 10),
                                        Lambda(x0!249,
                                        hr[x0!249] <= 12)),
                            Lambda(t!256,
                                   ForAll(t1!257,
                                        Implies(t!256 <=
                                        t1!257,
                                        hr[t1!257 + 1] ==
                                        If(complement(Lambda(x0!251,
                                        hr[x0!251] == 12))[t1!257],
                                        hr[t1!257] + 1,
                                        1)))))[t0!260],
               ForAll(t1!259,
                      Implies(t0!260 <= t1!259,
                              intersection(Lambda(x0!248,
                                        hr[x0!248] >= 10),
                                        Lambda(x0!249,
                                        hr[x0!249] <= 12))[t1!259]))))

In [ ]:
GRAMMAR = r"""
expr: op_expr

?op_expr: op_expr op op_expr   -> infix
        | op op_expr           -> prefix
        | op_expr op           -> postfix
        | op_expr "[" expr_list "]" -> index
        | op_expr UNDERSCORE identifier -> subscript
        | op_expr UNDERSCORE "<<" id_list ">>" -> subscript_tuple
        | op_expr EXCEPT except_clause ("," except_clause)* -> except_expr
        | term

term: if_expr
    | case_expr
    | quant_expr
    | choose_expr
    | atom

if_expr: "IF" expr "THEN" expr "ELSE" expr

case_expr: "CASE" case_arm+ ("OTHER" "->" expr)?
case_arm: expr "->" expr

quant_expr: quant_op quant_bounds ":" expr
quant_op: FORALL | EXISTS | FORALL2 | EXISTS2
quant_bounds: bound_list

choose_expr: "CHOOSE" choose_target (IN_OP expr)? ":" expr
choose_target: identifier | "<<" id_list ">>"

except_clause: "!"? except_lhs "=" expr
except_lhs: except_part+
except_part: "." identifier
           | "[" expr_list "]"

atom: identifier
    | identifier "(" arg_list? ")"
    | "(" expr ")"
    | "{" expr_list? "}"
    | "{" identifier IN_OP expr ":" expr "}"
    | "{" "<<" id_list ">>" IN_OP expr ":" expr "}"
    | "{" expr ":" bound_list "}"
    | "[" bound_list "|->" expr "]"
    | "[" expr "->" expr "]"
    | "<<" expr_list? ">>"
    | "[" expr_list "]"
    | "@"                       -> at_symbol
    | op                        -> op_symbol
    | NUMBER
    | STRING
    | "TRUE" -> true_
    | "FALSE" -> false_

arg_list: expr ("," expr)*
expr_list: expr ("," expr)*

bound_list: bound ("," bound)*
bound: identifier IN_OP expr
     | "<<" id_list ">>" IN_OP expr

id_list: identifier ("," identifier)+

identifier: NAME
name_list: NAME ("," NAME)*

op: OP | WORD_OP | IN_OP | BS_OP

NAME.1: /[A-Za-z0-9_][A-Za-z0-9_!]*/
NUMBER.2: /\\b[01]+|\\o[0-7]+|\\h[0-9A-Fa-f]+|[0-9]+(\\.[0-9]+)?/
STRING: /"([^"\\\\]|\\\\.)*"/

IN_OP.5: "\\in" | "∈"

EXCEPT.5: "EXCEPT"
UNDERSCORE.5: "_"
FORALL.5: "\\A"
EXISTS.5: "\\E"
FORALL2.5: "\\AA"
EXISTS2.5: "\\EE"
BS_OP.4: /\\[A-Za-z]+/
WORD_OP.4: /(SUBSET|UNION|DOMAIN|ENABLED|UNCHANGED)/


OP.2: /\[\]|::=|:=|<=>|=>|=<|<=|>=|\|->|->|\\|\\\/|\/\\|∧|∨|¬|≠|≤|≥|⇒|⇔|\.\.\.|\.\.|[+*'=<>~^|&%$#?!:.@\/\-]+/

%import common.WS
%ignore WS
"""

import lark


@lark.v_args(inline=True)
class TLATransformer(lark.Transformer):
    def expr(self, items):
        return items[0]
    def infix(self, left, op, right):
        return f"({left} {op} {right})"
    

lark.Lark(GRAMMAR, start="expr", parser="lalr", transformer=TLATransformer())


Lark(open('<string>'), parser='lalr', lexer='contextual', ...)